In [ ]:
!pip install boto3 botocore

In [ ]:
!pip list | grep -E "boto"

## Settings

Here we set up all the options for training.  Most are environment variables which will allow us to override values from pipelines and run this notebook with different setting such as the base model or number of training steps and learning rate.

In [ ]:
import os
from datetime import datetime

WORKING_DIR = os.environ.get("working_dir", f"/opt/app-root/src/pipelines-pvc/")
OUTPUT_DIR = os.path.join(os.getcwd(), f"{WORKING_DIR}/stable_diffusion_weights")
DATA_DIR = os.path.join(os.getcwd(), f"{WORKING_DIR}/data")

AWS_S3_PREFIX = os.environ.get("aws_s3_prefix", "text-to-image/stable_diffusion_weights")


## Save to S3

Now we have our model in a portable ONNX format, however, it's not doing much good in this notebook.  We need to push the model to our connected storage location.  Then we can use it in another notebook or serve the models for use within an application.

Note:  This requires a data connection to an S3 compatible bucket.  As part of the setup for this project, you added the [setup-s3.yaml](https://github.com/cfchase/text-to-image-demo/blob/main/setup/setup-s3.yaml) which created a local s3 bucket and data connections.

In [ ]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory_to_s3(local_directory, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)


In [ ]:
upload_directory_to_s3(OUTPUT_DIR, AWS_S3_PREFIX)

In [ ]:
list_objects(AWS_S3_PREFIX)